# Experiment 001: Baseline Evaluation

Evaluate the pre-optimized santa-2025.csv solution and calculate its score.

In [ ]:
import pandas as pd
import numpy as np
import math
from decimal import Decimal, getcontext

# Set high precision for calculations
getcontext().prec = 30

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
NV = 15

In [ ]:
def parse_value(s):
    """Parse a value that may be prefixed with 's'"""
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def get_poly(cx, cy, deg):
    """Get polygon vertices for a tree at position (cx, cy) with rotation deg"""
    rad = math.radians(deg)
    s, c = math.sin(rad), math.cos(rad)
    px, py = [], []
    for i in range(NV):
        x = TX[i] * c - TY[i] * s + cx
        y = TX[i] * s + TY[i] * c + cy
        px.append(x)
        py.append(y)
    return px, py

def get_bounding_box(px, py):
    """Get bounding box of a polygon"""
    return min(px), max(px), min(py), max(py)

def calculate_side_for_n(df, n):
    """Calculate the bounding box side length for n trees"""
    # Filter rows for this n
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    
    if len(rows) == 0:
        return None
    
    all_x, all_y = [], []
    for _, row in rows.iterrows():
        cx = parse_value(row['x'])
        cy = parse_value(row['y'])
        deg = parse_value(row['deg'])
        px, py = get_poly(cx, cy, deg)
        all_x.extend(px)
        all_y.extend(py)
    
    minx, maxx = min(all_x), max(all_x)
    miny, maxy = min(all_y), max(all_y)
    
    return max(maxx - minx, maxy - miny)

def calculate_total_score(df):
    """Calculate total score = sum(side^2 / n) for n=1 to 200"""
    total = 0.0
    sides = []
    for n in range(1, 201):
        side = calculate_side_for_n(df, n)
        if side is not None:
            score_n = (side ** 2) / n
            total += score_n
            sides.append((n, side, score_n))
    return total, sides

In [ ]:
# Load the pre-optimized solution
preopt_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df = pd.read_csv(preopt_path)
print(f"Loaded {len(df)} rows")
print(df.head(10))

In [ ]:
# Calculate the total score
total_score, sides = calculate_total_score(df)
print(f"\nTotal Score: {total_score:.6f}")
print(f"Target Score: 68.919154")
print(f"Gap: {total_score - 68.919154:.6f} ({(total_score - 68.919154) / 68.919154 * 100:.2f}%)")

In [ ]:
# Show the top 20 contributors to the score (highest score contribution)
sides_sorted = sorted(sides, key=lambda x: x[2], reverse=True)
print("\nTop 20 score contributors (n, side, score_contribution):")
for n, side, score_n in sides_sorted[:20]:
    print(f"  N={n:3d}: side={side:.6f}, contribution={score_n:.6f}")

In [ ]:
# Show small N values (1-10) - these have highest impact
print("\nSmall N values (1-10):")
for n, side, score_n in sides[:10]:
    print(f"  N={n:3d}: side={side:.6f}, contribution={score_n:.6f}")

In [ ]:
# Copy the pre-optimized solution as our baseline submission
import shutil
shutil.copy(preopt_path, '/home/submission/submission.csv')
print("Copied pre-optimized solution to /home/submission/submission.csv")

In [ ]:
# Verify the submission format
submission = pd.read_csv('/home/submission/submission.csv')
print(f"Submission shape: {submission.shape}")
print(f"Columns: {submission.columns.tolist()}")
print(submission.head())